In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/chest_xray/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = 'data/chest_xray'

Sto considerando di usare i seguenti modelli di ML:
1. ResNet.
2. GoogLeNet.
3. AlexNet.

In [ ]:
import os
import pathlib

data_dir = pathlib.Path(os.path.join(data_path, 'train'))
total = len(list(data_dir.glob('**/*.jpeg')))
bacteria = len(list(data_dir.glob('**/BACTERIA-*.jpeg')))
virus = len(list(data_dir.glob('**/VIRUS-*.jpeg')))
P = len(list(data_dir.glob('**/NORMAL-*.jpeg')))
N = total - P
print(f'#training set = {total}')
print(f'#positives = {P}')
print(f'#negatives = {N} (bacteria = {bacteria}, virus = {virus})')

#training set = 5232
#positives = 1349
#negatives = 3883 (bacteria = 2538, virus = 1345)


In [ ]:
import keras

training_set = utils.image_dataset_from_directory(
    data_dir,

)

Le immagini nel training set sono 5232 di cui: 1349 appartenenti alla classe NORMAL mentre 3883 alla classe PNEUMONIA.

```py
#training set = 5232
#positives = 1349
#negatives = 3883 (bacteria = 2538, virus = 1345)
```

Sto considerando diverse strategie per aumentare la dimensione del dataset:
1. Trasformazioni geometriche: specchiare, ruotare, tagliare, allungare.
   a. Specchiare e ruotare non mi sembrano che possano essere molto utili per questa tipologia di problema. Similmente anche
      il ritagliare l'immagine potrebbe rimuovere porzioni di immagini che potrebbero rivelarsi fondamentali per individuare o meno
      la polmonite.
   b. Tag
2. Modificare la luminosità.

È necessario ridimensionare le immagini per:
- Utilizzare meno risorse computazionali.
- Velocizzare sia addestramento che inferenza.
Al momento sto considerando le diverse strategie per il ridimensionamento delle immagini:
1. Ridimensionare tutte le immagini a 133x133 dove 133 è l'altezza minima tra tutte le immagini.
2. Ridimensionare tutte le immagini a 290x290 dove 290 è l'altezza media delle immagini

In [9]:
from keras import activations, layers, models

class ResBlock(layers.Layer):
    def __init__(self, k: (int, int), n: int, s: (int, int), **kwargs):
        super().__init__(**kwargs)
        self.conv1 = layers.Conv2D(n, k, s)
        self.bn1   = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(n, k, s)
        self.bn2   = layers.BatchNormalization()
        self.sum   = layers.Add()

    def __call__(self, x):
        z = self.conv1(x)
        z = self.bn1(z)
        z = activations.leaky_relu(z, 0.2)
        z = self.conv2(z)
        z = self.bn2(z)
        return self.sum(x, z)


class Resizer(layers.Layer):
    def __init__(self, target: (int, int), interpolation='bilinear', r: int=4, **kwargs):
        super().__init__(**kwargs)
        ## learnable layers
        self.conv1 = layers.Conv2D(16, (7, 7), activation=activations.leaky_relu)
        self.conv2 = layers.Conv2D(16, (1, 1), activation=activations.leaky_relu)
        self.bn1   = layers.BatchNormalization()
        self.r_blocks = [ResBlock((3, 3), 16, (1, 1)) for _ in range(r)]
        self.conv3 = layers.Conv2D(16, (3, 3), (1, 1))
        self.bn2   = layers.BatchNormalization()
        self.conv4 = layers.Conv2D(3, (7, 7), (1, 1))
        ## non-learnable layers
        self.sum  = layers.Add()
        self.sampler = layers.UpSampling2D(target, interpolation=interpolation)

    def __call__(self, x):
        y = self.conv1(x)
        y = self.conv2(y)
        y = self.bn1(y)
        y = self.sampler(y)
        z = y
        for r_block in self.r_blocks:
            z = r_block(z)
        z = self.conv3(z)
        z = self.bn2(z)
        z = self.sum(y, z)
        z = self.conv4(z)
        x = self.sampler(x)
        return self.sum(x, z)


In [10]:
from keras import activations, layers, models


def alex_net(resizer: layers.Layer):
    model = models.Sequential([
        resizer,
        layers.Conv2D(96, (11, 11), (4, 4),
                      activation=activations.relu),
        layers.BatchNormalization(),
        layers.MaxPool2D((3, 3), (2, 2)),
        layers.Conv2D(256, (5, 5), (2, 2), padding='same'),
        layers.BatchNormalization(),
        layers.MaxPool2D((3, 3), (2, 2)),
        layers.Conv2D(384, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(384, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.MaxPool2D((3, 3), (2, 2)),
        layers.Flatten(),
        layers.Dense(4096),
        layers.Dropout(0.5),
        layers.Dense(4096),
        layers.Dropout(0.5),
        layers.Dense(2, activation=activations.softmax)
    ])
    return model

In [13]:
resizer = Resizer((224, 244))
model = alex_net(resizer)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
hist = model.fit()